- In this chapter, we'll introduce **classification**
    - The **Logistic Regressor** shares the same foundations as the Linear Regressor

_____

# Defining a classification problem

- In the previous chapters, our regression model predicted a **continous** target variable
    - Classification, instead, predicts an outcome from a **finite set of possibilities**
    
- Simplest classification: binary
    - E.g. True/False, Non-default/default, etc.
        - If the prediction is True, we call it a **positive sample**. Otherwise, it's a **negative sample**

### Example binary classification problems

- Is this email spam?
- Is this house worth more than $200,000
- Will the customer open the coupon link in the email?
- Is the image of a man or a woman

- When there are more than two possible outcomes (i.e. the classification problem isn't binary), we call it a **multiclass classification**

### Example multiclass classification problems

- What kind of flower is this?
- What is the primary topic of the webpage?
- Which digit is drawn in the image?

____

# Formalization of the problem: binary classification

- We train our model on the vector $x$ which contains $n$ observations of the $k$ predictor variables and we map it to our outcome vector $y$. The $i^{th}$ observation can be written as:

# $(x_{i},y_{i}): x_{i} \in \Re^{k}, y_{i} \in \{0,1\}$

- Once we've trained our model, we'll have a classifier called a **classification function** of the form:

# $f: \Re^{k} \rightarrow \{0,1\}$